In [89]:
# Import required libraries
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import dash_table
import dash_leaflet as dl
import pandas as pd
import plotly.express as px
import urllib.parse
import base64;

# Import animal shelter CRUD module
from animal_shelter import AnimalShelter

# Set up the page layout
app = JupyterDash(__name__)

app.layout = html.Div([
       html.Form([
        html.Label('Username'),
        dcc.Input(id='input_user', type='text', placeholder='Enter username'),
        html.Label('Password'),
        dcc.Input(id='input_passwd', type='password', placeholder='Enter password'),
        html.Button('Submit', id='submit-btn', n_clicks=0)
    ]),
    # Add the "Grazioso Salvare Logo" image
    html.Img(src=f'data:image/png;base64,{encoded_image}',style={'width': '200px', 'height': '200px'}),

    
    html.Br(),
    html.H2("Animal Shelter CRUD Python Module - Beth Campbell"),
    
    

    html.Div(id='output'),

    html.Div([
        # Add the filter buttons
        html.Div([
            html.Button('Water Rescue', id='water-button'),
            html.Button('Mountain or Wilderness Rescue', id='mountain-button'),
            html.Button('Disaster or Individual Tracking Rescue', id='disaster-button')
        ]),

        # Add the data table and the pie chart
        html.Div([
            dash_table.DataTable(
                id='datatable-id',
                columns=[
                    {"name": i, "id": i, "deletable": False, "selectable": True} for i in ['breed', 'name', 'location_lat', 'location_long']
                ],
                data=[{'breed': '', 'name': '', 'location_lat': '', 'location_long': ''}],
                # Add filtering, sorting, and pagination for the data table
                filter_action='native',
                sort_action='native',
                page_action='native',
                page_current=0,
                page_size=10,
                # Add row selection to the data table
                row_selectable='single',
                style_cell={
                    'textAlign': 'left',
                    'font_family': 'Open Sans'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold',
                    'font_family': 'Open Sans'
                },
                style_data={
                    'whiteSpace': 'normal',
                    'height': 'auto',
                    'font_family': 'Open Sans'
                }
            ),

            # Add the pie chart and geolocation map below the data table
            html.Div([
                # Container div for geolocation map and pie chart
                html.Div([
                    html.Div([
                        # Add a container for the geolocation chart
                        html.Div(
                            id='map-id',
                            children=[
                                dl.Map(style={'width': '100%', 'height': '500px'},
                                       center=[30.75, -97.48],
                                       zoom=10,
                                       children=[
                                           dl.TileLayer(id='base-layer-id'),
                                           # Add a marker for the selected animal
                                           dl.Marker(id='animal-marker', position=[30.75, -97.48])
                                       ])
                                 ])
                    ], style={'display': 'inline-block', 'vertical-align': 'top', 'width': '50%'}),
                    html.Div([
                        # Add the pie chart
                        dcc.Graph(id='pie-chart')
                    ], style={'display': 'inline-block', 'vertical-align': 'top', 'width': '50%'})
                ], style={'display': 'flex', 'justify-content': 'space-between', 'margin-top': '20px'})
            ])
        ])
    ])
])

# Define callback to update authentication details and output-block
@app.callback(
    [Output('input_user', 'value'), Output('input_passwd', 'value'), Output('output', 'children')],
    [Input('submit-btn', 'n_clicks')],
    [State('input_user', 'value'), State('input_passwd', 'value')]
)
def update_auth_details_and_output_block(n_clicks, inputUser, inputPass):
    if n_clicks == 0:
        return ['', '', '']
    else:
        # Use URLLIB to set up the username and password cleanly
        # so that they can be passed to the MongoDB handler
        username = urllib.parse.unquote_plus(inputUser)
        password = urllib.parse.unquote_plus(inputPass)
        # Instantiate CRUD object with above authentication username and
        # password values
        shelter = AnimalShelter(username, password)
        # Perform a simple query on the data to check if the authentication worked
        query_result = shelter.read({'name': 'Ziggy'})
        # Check if the query result is not empty (assuming it's a list of documents)
        if query_result:
            # Join the query results into a single string with newline separators
            query_result_str = '\n'.join(map(str, query_result))
        else:
            query_result_str = 'No results found.'
        # Display query results
        return [inputUser, inputPass, query_result_str]

# Define the callback to retrieve the filtered data and update the data table
@app.callback(
    Output('datatable-id', 'data'),
    [Input('water-button', 'n_clicks'), Input('mountain-button', 'n_clicks'), Input('disaster-button', 'n_clicks')]
)
def update_data_table(water_clicks, mountain_clicks, disaster_clicks):
    # Define the filter dictionaries for each rescue type
    water_filter = {
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex": "Intact Female",
        "age_upon_outcome": {"$gte": 26, "$lte": 156},
    }

    mountain_filter = {
        "preferred_breeds": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "preferred_sex": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        "rescue_type": "Mountain or Wilderness"
    }

    disaster_filter = {
        "preferred_breeds": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "preferred_sex": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
        "rescue_type": "Disaster or Individual Tracking"
    }

    # Define the filter variable based on the button click
    if water_clicks:
        filtered_data = shelter.read(water_filter)
    elif mountain_clicks:
        filtered_data = shelter.read(mountain_filter)
    elif disaster_clicks:
        filtered_data = shelter.read(disaster_filter)
    else:
        # Return all shelter data if no button is clicked
        filtered_data = shelter.read({})
    
    # Check if the filtered data is not empty
    if filtered_data:
        # Create a DataFrame from the filtered data and select only the columns we need for the table and the geolocation chart
        filtered_data = pd.DataFrame.from_records(filtered_data)
        filtered_data = filtered_data[['breed', 'name', 'location_lat', 'location_long']]
    else:
        # Create an empty DataFrame with the same columns as the data table
        filtered_data = pd.DataFrame(columns=['breed', 'name', 'location_lat', 'location_long'])

    return filtered_data.to_dict('records')

# Define the callback to update the animal marker on the map
@app.callback(
    Output('animal-marker', 'position'),
    [Input('datatable-id', 'selected_row_ids')],
    [State('datatable-id', 'data')]
)
def update_animal_marker(selected_row_ids, rows):
    if selected_row_ids:
        # Find the selected row in the list of rows
        selected_row = next((row for row in rows if row['name'] == selected_row_ids[0]), None)
        # Update the marker position to the selected row's location
        if selected_row:
            return [selected_row['location_lat'], selected_row['location_long']]
    # Return the default location if no row is selected
    return [30.75, -97.48]


# Define the callback to generate the data for the pie chart and update the chart
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(data):
    # Create a pandas dataframe from the input data
    df = pd.DataFrame.from_records(data)
    # Check if the dataframe is not empty
    if not df.empty:
        # Group the data by breed and calculate the count
        grouped_data = df.groupby(['breed'])['name'].count().reset_index(name='count')
        # Create the pie chart figure
        fig = px.pie(grouped_data, values='count', names='breed')
    else:
        # Create an empty figure if the dataframe is empty
        fig = px.pie()
    # Return the figure
    return fig
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash app running on http://127.0.0.1:10971/
